IMPORTING MODULES

In [ ]:
import torch
from torch import nn

from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

from helper_functions import plot_decision_boundary

import matplotlib.pyplot as plt

HYPERPARAMETERS

In [ ]:
LERNING_RATE = 0.1
EPOCHS = 2000
INPUT_SHAPE = 2
HIDDEN_UNITS = 10
OUTPUT_SHAPE = 1
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
DEVICE

CREATING DATA

In [ ]:
n_samples = 1000

X, y = make_circles(n_samples=1000,
                    noise=0.03)
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

X.shape, y.shape

VISUALIZING DATA

In [ ]:
plt.scatter(x=X[:, 0],
            y=X[:, 1],
            c=y,
            cmap=plt.cm.RdYlBu)

CREATING A MODEL

In [ ]:
class CircleModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_shape,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=output_shape)
        )
    def forward(self, x):
        return self.layer_stack(x)
    
model = CircleModel(INPUT_SHAPE, HIDDEN_UNITS, OUTPUT_SHAPE).to(DEVICE)
model.state_dict()
        

TEST-TEST LOOP

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=LERNING_RATE)


for epoch in range(1, EPOCHS + 1):
    # TRAINING
    model.train()

    #1 Forward pass
    train_logits = model(X_train).squeeze()
    train_pred = torch.round(torch.sigmoid(train_logits))

    #2 Calculate loss
    train_loss = loss_fn(train_logits, y_train)

    #3 Optimizer zero grad
    optimizer.zero_grad()

    #4 Loss backward
    train_loss.backward()

    #5 Optimizer step
    optimizer.step()

    # TESTING
    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fn(test_pred, y_test)
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.2f} | Test loss: {test_loss:.2f}")


In [ ]:
model.eval()
with torch.inference_mode():
    preds = torch.round(torch.sigmoid(model(X_test))).squeeze()

plt.figure(figsize=(5, 5))
plt.title("Test")
plot_decision_boundary(model, X_test, preds)